In [1]:
import numpy as np

In [4]:
N=6
C = np.array([
    [7+5*np.sin(i)*np.cos(j) for j in range(N)]
    for i in range(N)
])
C = C.round(2)

array([[  7.  ,   7.  ,   7.  ,   7.  ,   7.  ,   7.  ],
       [ 11.21,   9.27,   5.25,   2.83,   4.25,   8.19],
       [ 11.55,   9.46,   5.11,   2.5 ,   4.03,   8.29],
       [  7.71,   7.38,   6.71,   6.3 ,   6.54,   7.2 ],
       [  3.22,   4.96,   8.57,  10.75,   9.47,   5.93],
       [  2.21,   4.41,   9.  ,  11.75,  10.13,   5.64]])

In [14]:
min1 = np.min(C, axis=1).reshape((-1,1))
min1.round(2)

array([[ 7.  ],
       [ 2.83],
       [ 2.5 ],
       [ 6.3 ],
       [ 3.22],
       [ 2.21]])

In [15]:
C2 = C - min1
C2 = C2.round(2)
C2

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 8.37,  6.44,  2.41,  0.  ,  1.42,  5.36],
       [ 9.05,  6.96,  2.61,  0.  ,  1.53,  5.79],
       [ 1.4 ,  1.08,  0.4 ,  0.  ,  0.24,  0.9 ],
       [ 0.  ,  1.74,  5.36,  7.53,  6.26,  2.71],
       [ 0.  ,  2.2 ,  6.79,  9.54,  7.93,  3.43]])

In [22]:
min2 = np.min(C2, axis=0).round(2)
min2

array([ 0.,  0.,  0.,  0.,  0.,  0.])

In [95]:
l1 = min1.sum().round(2)
l1

24.059999999999999

In [25]:
import itertools

In [40]:
def heuristic(C):
    S = np.zeros(C.shape)
    for i, j in itertools.product(*[range(d) for d in C.shape]):
        if C[i,j].round(2) != 0:
            S[i,j] = -np.infty
            continue
        mr, mc = 2*[np.infty]
        for i1 in range(C.shape[0]):
            if i1 == i: continue
            mc = min(mc, C[i1, j])
        for j1 in range(C.shape[1]):
            if j1 == j: continue
            mr = min(mr, C[i, j1])
        S[i,j] = mr + mc
    return S

In [41]:
H2 = heuristic(C2)
H2

array([[ 0.  ,  1.08,  0.4 ,  0.  ,  0.24,  0.9 ],
       [ -inf,  -inf,  -inf,  1.42,  -inf,  -inf],
       [ -inf,  -inf,  -inf,  1.53,  -inf,  -inf],
       [ -inf,  -inf,  -inf,  0.24,  -inf,  -inf],
       [ 1.74,  -inf,  -inf,  -inf,  -inf,  -inf],
       [ 2.2 ,  -inf,  -inf,  -inf,  -inf,  -inf]])

In [42]:
B1u = 7+2.83+6.71+4.3+5.93+3.21
B1u, round(B1, 2)

(26.77, 26.77)

In [50]:
C2

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 8.37,  6.44,  2.41,  0.  ,  1.42,  5.36],
       [ 9.05,  6.96,  2.61,  0.  ,  1.53,  5.79],
       [ 1.4 ,  1.08,  0.4 ,  0.  ,  0.24,  0.9 ],
       [ 0.  ,  1.74,  5.36,  7.53,  6.26,  2.71],
       [ 0.  ,  2.2 ,  6.79,  9.54,  7.93,  3.43]])

In [60]:
C_2_1 = C2[np.array([1,2,3,4,5])-1, :]
C_2_1 = C_2_1[:,np.array([2,3,4,5,6])-1]
C_2_1[0,-1] = np.infty
C_2_1

array([[ 0.  ,  0.  ,  0.  ,  0.  ,   inf],
       [ 6.44,  2.41,  0.  ,  1.42,  5.36],
       [ 6.96,  2.61,  0.  ,  1.53,  5.79],
       [ 1.08,  0.4 ,  0.  ,  0.24,  0.9 ],
       [ 1.74,  5.36,  7.53,  6.26,  2.71]])

In [66]:
C_2_1_mr = np.min(C_2_1, axis=1).reshape((-1,1))
C_2_1_mr

array([[ 0.  ],
       [ 0.  ],
       [ 0.  ],
       [ 0.  ],
       [ 1.74]])

In [63]:
C_2_1_cor = C_2_1 - C_2_1_mr
C_2_1_cor

array([[ 0.  ,  0.  ,  0.  ,  0.  ,   inf],
       [ 6.44,  2.41,  0.  ,  1.42,  5.36],
       [ 6.96,  2.61,  0.  ,  1.53,  5.79],
       [ 1.08,  0.4 ,  0.  ,  0.24,  0.9 ],
       [ 0.  ,  3.62,  5.79,  4.52,  0.97]])

In [67]:
C_2_1_mc = C_2_1_cor.min(axis=0)
C_2_1_mc

array([ 0. ,  0. ,  0. ,  0. ,  0.9])

In [68]:
C_2_1_cor2 = C_2_1_cor - C_2_1_mc
C_2_1_cor2

array([[ 0.  ,  0.  ,  0.  ,  0.  ,   inf],
       [ 6.44,  2.41,  0.  ,  1.42,  4.46],
       [ 6.96,  2.61,  0.  ,  1.53,  4.89],
       [ 1.08,  0.4 ,  0.  ,  0.24,  0.  ],
       [ 0.  ,  3.62,  5.79,  4.52,  0.07]])

In [97]:
l2_1 = C_2_1_mr.sum() + C_2_1_mc.sum()
l2_1

2.6400000000000001

In [98]:
round(l1 + l2_1, 2)

26.699999999999999

In [70]:
26.77+3.21

29.98

In [72]:
round(C_2_1_mr.sum() + C_2_1_mc.sum(),2)

2.6400000000000001

In [73]:
H21 = heuristic(C_2_1_cor2)
H21

array([[ 0.  ,  0.4 ,  0.  ,  0.24,  -inf],
       [ -inf,  -inf,  1.42,  -inf,  -inf],
       [ -inf,  -inf,  1.53,  -inf,  -inf],
       [ -inf,  -inf,  0.  ,  -inf,  0.07],
       [ 0.07,  -inf,  -inf,  -inf,  -inf]])

In [76]:
def row_transform(C):
    mins = np.min(C, axis=1).reshape((-1,1))
    return C - mins, mins

def col_transform(C):
    mins = np.min(C, axis=0)
    return C - mins, mins

In [99]:
C_2_2 = C2.copy()
C_2_2[6-1,1-1] = np.infty
C_2_2

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 8.37,  6.44,  2.41,  0.  ,  1.42,  5.36],
       [ 9.05,  6.96,  2.61,  0.  ,  1.53,  5.79],
       [ 1.4 ,  1.08,  0.4 ,  0.  ,  0.24,  0.9 ],
       [ 0.  ,  1.74,  5.36,  7.53,  6.26,  2.71],
       [  inf,  2.2 ,  6.79,  9.54,  7.93,  3.43]])

In [100]:
C_2_2, C_2_2_mr = row_transform(C_2_2)
C_2_2, C_2_2_mc = col_transform(C_2_2)
C_2_2 = C_2_2.round(2)
C_2_2

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 8.37,  6.44,  2.41,  0.  ,  1.42,  5.36],
       [ 9.05,  6.96,  2.61,  0.  ,  1.53,  5.79],
       [ 1.4 ,  1.08,  0.4 ,  0.  ,  0.24,  0.9 ],
       [ 0.  ,  1.74,  5.36,  7.53,  6.26,  2.71],
       [  inf,  0.  ,  4.59,  7.34,  5.73,  1.23]])

In [105]:
l2_2 = C_2_2_mr.sum() + C_2_2_mc.sum()
l2_2.round(2)

2.2000000000000002

In [102]:
C_2_2_mr, C_2_2_mc

(array([[ 0. ],
        [ 0. ],
        [ 0. ],
        [ 0. ],
        [ 0. ],
        [ 2.2]]), array([ 0.,  0.,  0.,  0.,  0.,  0.]))

In [106]:
l1 + l2_2

26.259999999999998